In [49]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from splinter.exceptions import ElementDoesNotExist
import re
import requests

In [50]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## Scrape Craigslist Freestuff

In [53]:
# Go to sacramento cl and navigate to 1st item

url = "https://sacramento.craigslist.org/d/free-stuff/search/zip"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
first_result = soup.find('li', class_="result-row")
browser.visit(first_result.a['href'])

In [54]:
# Using Splinter to scrape cl and store data in dataframe stuff (clean up condition)

stuff = pd.DataFrame(columns=['title','age', 'location', 'condition'])

for x in range(1, 4):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find(id="titletextonly").text
    age = soup.find('time', class_="date timeago")["datetime"]
    loc = soup.find(id='map')
    coord = [loc["data-latitude"], loc["data-longitude"]]
    condit = soup.find('p', class_="attrgroup")
    cond = ''
    if (condit): cond = condit.b.text
    
    stuff.loc[x] = [title, age, coord, cond]
    browser.click_link_by_partial_text('next')

print(stuff)

                               title                       age  \
1  Free- 7 - 9 feet of aluminum duct  2019-10-19T12:44:10-0700   
2                          bookshelf  2019-10-19T12:41:40-0700   
3                  Wood stove Insert  2019-10-19T12:34:55-0700   

                   location condition  
1  [38.568300, -121.436600]            
2  [38.568300, -121.436600]      good  
3  [38.924326, -121.054562]            
